### Este notebook contem todo o processo de mineração do dataset de casas em King County, EUA

#### Carregamento dos dados

In [1]:
from pysrc.loading import *

In [2]:
dataframe.sample(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
17717,1018000276,20150327T000000,217000,3,2.50,1340,4200,2.0,0,0,...,7,1340,0,2002,0,98002,47.2942,-122.226,990,4520
13711,4039500610,20140820T000000,440000,3,1.75,1430,8400,1.0,0,0,...,7,1430,0,1961,0,98008,47.6073,-122.127,1570,7800
4737,3629920300,20141103T000000,425000,3,2.25,1260,3000,2.0,0,0,...,7,1260,0,2003,0,98029,47.5454,-121.997,1630,3042
4242,925059198,20140828T000000,514000,3,2.00,1770,7200,1.0,0,0,...,7,1770,0,1967,0,98033,47.6642,-122.173,2250,11250
16208,2557000380,20140618T000000,287500,4,2.50,2570,9000,1.0,0,0,...,8,1590,980,1979,0,98023,47.2986,-122.372,2120,8571


In [3]:
# Colunas da base de dados, inicialmente
showInitialCols()

21 colunas
['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


## Preprocessamento

In [4]:
from pysrc.preprocessing import *

#### Remover colunas que não nos interessam

As colunas removidas são impressas abaixo

In [5]:
dropUselessCols()

In [6]:
# Colunas restantes
showFinalCols()

16 colunas restantes
Removidas  ['id', 'date', 'zipcode', 'lat', 'long']
['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']


#### Verificar qualidade dos dados restantes

In [7]:
# Checando premissas fundamentais sobre os atributos
checkDataRanges()

Zero valores faltantes
Todos os valores sao nao-negativos
Anos de contrucao entre 1900 e 2015
Ano de renovacao >= ano de contrucao


#### Aplicar algumas tranformações convenientes

In [8]:
# Agora precisamos transformar os atributos "bathrooms" e "floors" inteiros


In [9]:
# Categorizar tamanhos
